#  **TP : Analyse des dépondances conjointes**


Ce travail pratique se déroule en deux étapes complémentaires :

1. Mise en situation guidée : vous appliquerez, à travers un exemple détaillé, les principales techniques d’analyse de dépendance conjointe afin de vous familiariser avec leur logique et leur mise en œuvre.

2. Étude de cas appliquée : vous serez ensuite amenés à traiter un problème plus complexe portant sur l’analyse de logs de sécurité, relatifs aux tentatives de connexion au système d’information d’une entreprise. L’objectif est de mobiliser les méthodes vues en première partie pour en extraire des informations pertinentes et évaluer d’éventuelles dépendances significatives.

# **Première partie: Travail à faire**

Cette première partie montre étape par étape comment préparer et analyser un dataset pour appliquer une **régression linéaire**.

Nous allons :
- Créer un dataset réaliste (attaques, protocoles, etc.)
- Encodage des variables qualitatives
- Calculer des corrélations numériques
- Faire une analyse de la variance (ANOVA) avec Boxplot
- Faire un test du Khi-deux
- Construire un modèle de régression linéaire
- Interpréter les coefficients et visualiser les résultats

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import chi2_contingency, f_oneway

ModuleNotFoundError: No module named 'sklearn'

## Étape 1 : Création du dataset

In [ ]:
data = {
    "Attack_Type": [
        "Phishing","Malware","DDoS","PortScan","Ransomware","Phishing",
        "Malware","SQLInjection","DDoS","Phishing","Ransomware","PortScan"
    ],
    "Protocol": [
        "TCP","UDP","ICMP","TCP","TCP","UDP",
        "UDP","TCP","ICMP","TCP","UDP","TCP"
    ],
    "Source_Country": [
        "US","RU","CN","US","IR","FR",
        "RU","IN","CN","BR","RU","US"
    ],
    "Target_OS": [
        "Windows","Linux","Windows","Linux","Windows","MacOS",
        "Linux","Windows","Linux","Windows","MacOS","Linux"
    ],
    "Packets": [120, 300, 5000, 150, 400, 220, 330, 800, 7000, 130, 600, 200],
    "Bytes":   [24000, 60000, 1200000, 45000, 80000, 50000, 70000, 120000, 2000000, 30000, 90000, 40000],
    "Duration": [15, 30, 120, 10, 50, 25, 35, 60, 200, 20, 90, 15],
    "Severity": [
        "High","Medium","High","Low","Critical","Medium",
        "Medium","High","Critical","Low","Critical","Low"
    ],
    "Response_Time": [45, 60, 200, 30, 180, 50, 70, 120, 300, 40, 220, 35]
}

df = pd.DataFrame(data)
df.head()

## Étape 2 : Corrélations numériques

In [ ]:
corr = df[["Packets","Bytes","Duration","Response_Time"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Corrélations numériques avec Response_Time")
plt.show()

## Étape 3 : Analyse de la variance (ANOVA)

In [ ]:
groups = [df.loc[df["Severity"]==lvl, "Response_Time"].values 
          for lvl in df["Severity"].unique()]
F, p = f_oneway(*groups) # f_oneway permet de faire l'ANOVA
print("ANOVA F:", round(F, 3), "| p-value:", round(p, 4))

sns.boxplot(x="Severity", y="Response_Time", data=df)
plt.title("Response_Time vs Severity (Boxplot)")
plt.show()

## Étape 4 : Test du Khi-deux

In [ ]:
tab = pd.crosstab(df["Attack_Type"], df["Severity"])
chi2, p, ddl, expected = chi2_contingency(tab)
print("Chi²:", round(chi2, 3), "| p-value:", round(p, 4))
pd.DataFrame(expected, index=tab.index, columns=tab.columns)

## Étape 5 : Préparation des données pour la régression

In [ ]:
num_cols = ["Packets","Bytes","Duration"]
cat_cols = ["Attack_Type","Protocol","Source_Country","Target_OS","Severity"]

X = df[num_cols + cat_cols]
y = df["Response_Time"]

preproc = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_cols)
])

## Étape 6 : Régression linéaire

In [ ]:
pipe = Pipeline(steps=[
    ("prep", preproc),
    ("reg", LinearRegression())
])

pipe.fit(X, y)
y_pred = pipe.predict(X)

print("R²:", round(r2_score(y, y_pred), 3))
print("RMSE:", round(np.sqrt(mean_squared_error(y, y_pred)), 3))

## Étape 7 : Analyse des coefficients

In [ ]:
ohe = pipe.named_steps["prep"].named_transformers_["cat"]
ohe_features = ohe.get_feature_names_out(cat_cols)
all_features = num_cols + list(ohe_features)

coef = pd.Series(pipe.named_steps["reg"].coef_, index=all_features).sort_values(ascending=False)

print("Intercept:", round(pipe.named_steps["reg"].intercept_, 3))
print("\nCoefficients :")
print(coef.round(3))

## Étape 8 : Visualisation des prédictions

In [ ]:
plt.scatter(y, y_pred, color="blue")
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("Response_Time réel")
plt.ylabel("Response_Time prédit")
plt.title("Régression linéaire : Réel vs Prédit")
plt.show()

# **Deuxième partie: Travail à faire**

**Contexte du TP**

L’entreprise CyberCorp enregistre dans son SIEM (Security Information and Event Management) toutes les tentatives de connexion à son système d’information (voir fichier `cybersecurity_latency.csv`).
Chaque log contient :

- **user_id** : identifiant utilisateur (user_1, user_2, …)

- **role** : Admin, Employe, Manager, Externe

- **department** : IT, Finance, RH, Production

- **hour** : heure de connexion (0–23h)

- **location** : pays d’origine (Maroc, France, USA, Chine, Russie)

- **device_type** : Desktop, Mobile, Serveur

- **failed_attempts_24h** : nombre d’échecs d’authentification des dernières 24h

- **auth_success** : succès (1) ou échec (0) de l’authentification

- **latency_ms** : latence de la connexion en millisecondes

- **anomaly_score** : score d’anomalie (entre 0 et 1)

Vous devez étudier la variable latency_ms (latence réseau) en fonction des autres caractéristiques, à travers les mesures de dépondances et un modèle de regression linéaire.

Faites le travail selon les étapes ci-après:

## Étape 1 : Chargement et exploration

- Importer les bibliothèques : pandas, numpy, matplotlib, seaborn, sklearn.

- Générer ou charger le dataset fourni.

- Afficher les 10 premières lignes et vérifier les types de variables.

## Étape 2 : Analyse exploratoire

- Faire un heatmap des corrélations entre variables numériques (failed_attempts_24h, hour, latency_ms, anomaly_score).

- Faire un boxplot de `latency_ms` selon `location`.

- Faire un test ANOVA pour voir si `latency_ms` varie selon `device_type`.

- Faire un test du Khi-deux pour tester la dépendance entre `role` et `auth_success`.

## Étape 3 : Préparation des données

- Séparer les variables explicatives (X) et la cible (y = `latency_ms`).

- Identifier les colonnes numériques et catégorielles.

- Utiliser un ColumnTransformer avec :

    - **StandardScaler()** pour les variables numériques

    - **OneHotEncoder(drop="first")** pour les variables catégorielles

## Étape 4 : Modèle de régression linéaire

- Construire un pipeline avec :

    - préprocesseur (ColumnTransformer)

    - modèle (LinearRegression)

- Entraîner le modèle et faire les prédictions.

- Calculer les métriques :

    - R² (coefficient de détermination)

    - RMSE (erreur quadratique moyenne)

## Étape 5 : Interprétation

- Afficher les coefficients associés aux variables.

- Identifier :

    - Quelles localisations ou quels types de device augmentent la latence.

    - L’effet des variables quantitatives (failed_attempts_24h, hour) sur la latence.

## Étape 6 : Effet interaction entre deux variables

**Exemple illustratif**: On veut prédire la probabilité d’échec de connexion (failed_attempts) en fonction de :

- `device` (PC, serveur, smartphone),
- `hour` (heure de la journée).

Le modèle:  **Y=β0​+β1​⋅device+β2​⋅hour+ε** suppose que **l’effet du type d’appareil (device) est le même quelle que soit l’heure (hour)**.

Pourtant, l’effet du device à changer selon l’heure, c'est ce qu'on appelle **effet interaction**. Par exemple :

- Les serveurs peuvent avoir plus d’échecs de connexion la nuit (maintenance, scans automatiques).
- Les smartphones peuvent avoir plus d’échecs en journée (mauvais réseaux).

Une interaction entre deux variables signifie que l’effet d’une variable sur la variable cible dépend de la valeur de l’autre variable.

Le modèle avec interaction va capturer cette nuance :  **Y=β0​+β1​⋅device+β2​⋅hour+β3​⋅(device×hour)+ε**

**Question**: Evaluer l'effet interaction entre `devices=="Serveur` et `failed_attempts` sur le modèle de regression